Saving and loading
==================

Contrarily to Matlab with its `.mat` file, python does not come with a single standard way to save data. To ease saving and sharing data, Kinetics Toolkit provides [two functions](api/ktk.loadsave.rst) that load and save its own `.ktk.zip` format.

Let's create a dummy variable with some stuff in it.

In [1]:
import numpy as np
import ktk
import zipfile
import json

ktk.lab()

variable1 = {'some_array': np.arange(0, 5, 0.5),
             'some_timeseries': ktk.TimeSeries(
                 time=np.arange(10),
                 data={
                     'data1': np.arange(10),
                     'data2': np.arange(10) ** 2
                 })
            }

Saving a variable
-----------------

In [2]:
ktk.save('filename.ktk.zip', variable1)

Loading a variable
------------------

In [3]:
variable2 = ktk.load('filename.ktk.zip')

As expected, `variable2` is equal to `variable1`:

In [4]:
variable1

{
           'some_array': <array of shape (10,)>,
      'some_timeseries': TimeSeries with attributes: data: <dict with 2 entries>, ...
}

In [5]:
variable2

{
           'some_array': <array of shape (10,)>,
      'some_timeseries': TimeSeries with attributes: data: <dict with 2 entries>, ...
}

Supported types
---------------

`ktk.save` supports any combination of the following types:

- dict containing any supported class
- list containing any supported class
- str
- int
- float
- True
- False
- None
- numpy.array
- pandas.DataFrame
- pandas.Series
- ktk.TimeSeries

The .ktk.zip format
-------------------

The `.ktk.zip` format is built to be as portable and simple as possible. It is a standard zip file that contains two JSON files:

- metadata.json: The file metadata such as the save date, the computer's operating system, etc.
- data.json: The data saved as a standard json file. The data types that are not supported natively by the JSON file format (e.g., numpy, pandas and ktk objects) are converted to supported objects so that they are fully readable in other environments such as Matlab.

For example, let's examine the file we just saved:

In [6]:
archive = zipfile.ZipFile('filename.ktk.zip', 'r')
print(archive.read('metadata.json').decode())

{
	"Software": "Kinetics Toolkit (ktk)",
	"Version": "master",
	"Computer": "Mac",
	"FileFormat": 1.0,
	"SaveDate": "2020-10-09",
	"SaveTime": "20:35:18",
	"User": "felix"
}


In [7]:
print(archive.read('data.json').decode())

{
	"some_array": {
		"class__": "numpy.array",
		"value": [
			0.0,
			0.5,
			1.0,
			1.5,
			2.0,
			2.5,
			3.0,
			3.5,
			4.0,
			4.5
		]
	},
	"some_timeseries": {
		"class__": "ktk.TimeSeries",
		"time": [
			0,
			1,
			2,
			3,
			4,
			5,
			6,
			7,
			8,
			9
		],
		"time_info": {
			"Unit": "s"
		},
		"data_info": {},
		"data": {
			"data1": [
				0,
				1,
				2,
				3,
				4,
				5,
				6,
				7,
				8,
				9
			],
			"data2": [
				0,
				1,
				4,
				9,
				16,
				25,
				36,
				49,
				64,
				81
			]
		},
		"events": []
	}
}


Loading a .ktk.zip in Matlab
----------------------------

    % Create a temporary folder to unzip to
    mkdir('temp');
    unzip(filename, 'temp');

    % Load the file contents
    data = jsondecode(fileread('temp/data.json'));
    metadata = jsondecode(fileread('temp/metadata.json'));

Since the types are not the same between python, JSON and Matlab, here is how Matlab will reconstruct the following python types:

| Python                           | Matlab                       |
|----------------------------------|------------------------------|
| None                             | NaN                          |
| True, False                      | true, false                  |
| int, float                       | double                       |
| str                              | char                         |
| dict                             | struct                       |
| list of different data types     | cell array                   |
| list of boolean                  | array of logical             |
| list of int or float             | array of double              |
| list of str                      | cell array of char           |
| list of dict, same field names   | structure array              |
| list of dict, diff field names   | cell array of struct         |
| numpy.array                      | array of double              |
| ktk.TimeSeries                   | struct                       |
| complex                          | struct with real and imag    |


    
*tuples are saved as lists in JSON.

For more information on saving and loading, please check the [API Reference](api/ktk.loadsave.rst).